In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from attacks.minimization import *
import json
import os
import torchattacks
import time

from _testbed_utils import original_dataloader, get_data
from models import create_simple_neural_network, create_dynamic_neural_network, create_dynamic_dm_neural_network
from dataloader_ids import load_and_prepare_data

from sklearn.metrics import precision_score, recall_score, f1_score

torch.set_printoptions(sci_mode=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Attack

In [ ]:
def get_data(dataset_key, encoding_key, multiclass, seed=42):
    train_loader, _, test_loader, test_indices, input_dim, output_dim, y_mapping, scaler, _ = load_and_prepare_data(dataset_key, encoding_key, multiclass, random_state=seed)
    return train_loader, test_loader, test_indices, input_dim, output_dim, y_mapping, scaler

def extract_hidden_dims(params):
    hidden_dims = []
    for key, value in params.items():
        if key.startswith("hidden_dim_layer_"):
            hidden_dims.append(value)

    # Sort by layer index in case keys are unordered
    hidden_dims = [v for k, v in sorted((key, value) for key, value in params.items() if key.startswith("hidden_dim_layer_"))]
    return hidden_dims

def get_and_create_model(dataset, encoding, multiclass, input_dim, output_dim):
    
    best_params_fp = f"results/model_discovery/{dataset}/{encoding}/best_params.json"

    with open(best_params_fp, "r") as f:
        best_params = json.load(f)
        hidden_dims = extract_hidden_dims(best_params)
        learning_rate = best_params["lr"]
        dropout_rate = best_params["dropout"]
    
    if encoding == 'DM':
        model, _, _ = create_dynamic_dm_neural_network(
            input_dim=input_dim,
            output_dim=output_dim,
            multiclass=multiclass,
            hidden_dims=hidden_dims,
            optimizer="adam",
            lr=learning_rate,
            dropout_rate=dropout_rate
        )
    else:
        model, _, _ = create_dynamic_neural_network(
            input_dim=input_dim,
            output_dim=output_dim,
            multiclass=multiclass,
            hidden_dims=hidden_dims,
            optimizer="adam",
            lr=learning_rate,
            dropout_rate=dropout_rate
        )
    model = model.to(device)
    return model

def get_latest_model_state(dataset, encoding, mode, multiclass, input_dim, output_dim):
    model = get_and_create_model(dataset, encoding, multiclass, input_dim, output_dim)

    model_dir = f"trained_models/{dataset}/{encoding}/{mode}/"

    model_files = [f for f in os.listdir(model_dir) if f.endswith(".pth")]
    if not model_files:
        raise FileNotFoundError("No model files found")
    latest_model = max(model_files, key=lambda f: os.path.getmtime(os.path.join(model_dir, f)))
    latest_model_path = os.path.join(model_dir, latest_model)
    print(f"Loading checkpoint: {latest_model_path}")

    if encoding == 'DM':
        state = torch.load(latest_model_path, map_location='cpu')
        model.load_state_dict(state, strict=False)
    else:
        model.load_state_dict(torch.load(latest_model_path, map_location=torch.device('cpu')))
        
    model.eval()
    return model                           

In [ ]:
def load_model_and_data(dataset, encoding, training_mode, multiclass, seed):
    """
    Load the trained model and test data for a given dataset and encoding.

    Args:
      dataset (str): dataset name (e.g., 'mirai').
      encoding (str): feature encoding type ('Raw', 'DM', etc.).
      multiclass (bool): whether the task is multiclass.

    Returns:
      model (torch.nn.Module): loaded PyTorch model in eval mode.
      all_data (torch.Tensor): concatenated test inputs (n_samples, n_features).
      all_labels (torch.Tensor): concatenated test labels (n_samples,).
      df_clean (pd.DataFrame): clean test inputs as a DataFrame.
    """
    # training_mode = 'baseline_True' if encoding in ('DM', 'Stats') else 'baseline'
    _, test_loader, _, input_dim, output_dim, _, scaler = get_data(
        dataset, encoding, multiclass, seed=seed
    )
    model = get_latest_model_state(
        dataset, encoding, training_mode, multiclass, input_dim, output_dim
    ).to(device)
    model.eval()

    all_data, all_labels = [], []
    for x, y in test_loader:
        all_data.append(x)
        all_labels.append(y)

    
    all_data = torch.cat(all_data, dim=0).to(device)
    all_labels = torch.cat(all_labels, dim=0).to(device)

    df_clean = pd.DataFrame(
        all_data.cpu().numpy(),
        columns=[f"feature_{i}" for i in range(all_data.size(1))]
    )
    print(df_clean.shape)
    return model, all_data, all_labels, df_clean

def perform_attack(model, data, labels, method, params):
    """
    Generate adversarial examples using a specified attack.

    Args:
      model (torch.nn.Module): the target model.
      data (torch.Tensor): clean inputs (n_samples, n_features).
      labels (torch.Tensor): true labels (n_samples,).
      method (str): one of 'fgsm', 'cw', 'deepfool', 'jsma'.
      params (dict): attack-specific parameters:
        - fgsm: {'eps': float}
        - cw: {'c': float, 'kappa': float}
        - jsma: {'theta': float, 'gamma': float}
        - deepfool: {}
    Returns:
      adv_np (np.ndarray): adversarial examples as NumPy array.
    """
    if method == 'fgsm':
        atk = torchattacks.FGSM(model, eps=params['eps'])
        adv = atk(data, labels)
    elif method == 'cw':
        atk = torchattacks.CW(model, c=params['c'], kappa=params['kappa'], steps=100, lr=0.05)
        adv = atk(data, labels)
    elif method == 'deepfool':
        atk = torchattacks.DeepFool(model, overshoot=params['overshoot'], steps=20)
        adv = atk(data, labels)
    elif method == 'jsma':
        atk = torchattacks.JSMA(model, theta=params['theta'], gamma=params['gamma'])
        adv = atk(data, labels)
    else:
        raise ValueError(f"Unknown attack: {method}")
    return adv.cpu().numpy()

def fraction_clipped(adv_np):
    """
    Compute the fraction of features that are exactly 0 or 1 after clipping.

    Args:
      adv_np (np.ndarray): adversarial inputs (n_samples, n_features).

    Returns:
      float: fraction of entries equal to 0 or 1.
    """
    n, f = adv_np.shape
    clipped = np.logical_or(adv_np == 0, adv_np == 1).sum()
    return clipped / (n * f)

def evaluate(model, adv_np, df_clean, labels, encoding):
    """
    Evaluate model performance and perturbation norms on adversarial data.

    Args:
      model (torch.nn.Module): the target model.
      adv_np (np.ndarray): adversarial inputs (n_samples, n_features).
      df_clean (pd.DataFrame): clean inputs as DataFrame.
      labels (torch.Tensor): true labels (n_samples,).
      encoding (str): 'Raw' or other encoding type.

    Returns:
      acc (float): accuracy.
      prec (float): weighted precision.
      rec (float): weighted recall.
      f1 (float): weighted F1 score.
      mean_norm (float): mean L2 (raw) or Frobenius (DM) perturbation norm.
      clip_frac (float): fraction of features clipped to 0 or 1.
    """
    X_adv_t = torch.from_numpy(adv_np).float().to(device)
    outputs = model(X_adv_t)
    _, preds = torch.max(outputs, 1)

    y_true = labels.cpu().numpy()
    y_pred = preds.cpu().numpy()
    acc  = (y_pred == y_true).mean()
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec  = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1   = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    # Compute clean norms for each norm type
    clean_linf = np.linalg.norm(df_clean.values, ord=np.inf, axis=1)
    clean_l2   = np.linalg.norm(df_clean.values, ord=2, axis=1)
    clean_l0   = np.linalg.norm(df_clean.values, ord=0, axis=1)

    # Compute median reference for each norm
    ref_linf = np.median(clean_linf)
    ref_l2   = np.median(clean_l2)
    ref_l0   = np.median(clean_l0)

    # Compute the perturbation delta
    delta = adv_np - df_clean.values

    # Compute mean norms for perturbation
    linf_mean_norm = np.linalg.norm(delta, ord=np.inf, axis=1).mean()
    l2_mean_norm   = np.linalg.norm(delta, ord=2, axis=1).mean()
    l0_mean_norm   = np.linalg.norm(delta, ord=0, axis=1).mean()

    # Compute percentage changes
    pct_change_linf = 100 * linf_mean_norm / ref_linf
    pct_change_l2   = 100 * l2_mean_norm   / ref_l2
    pct_change_l0   = 100 * l0_mean_norm   / ref_l0

    mean_norms = {
        'linf': linf_mean_norm,
        'l2': l2_mean_norm,
        'l0': l0_mean_norm
    }
    pct_changes = {
        'linf': pct_change_linf,
        'l2': pct_change_l2,
        'l0': pct_change_l0
    }

    clip_frac = fraction_clipped(adv_np)
    return acc, prec, rec, f1, mean_norms, clip_frac, pct_changes

def sweep_attack(dataset, attack, encoding, model, all_data, all_labels, df_clean):
    """
    Run attack over various parameter combinations and collect metrics.
    
    Args:
      attack (str): attack type ('fgsm', 'cw', 'jsma', etc.).
      dataset (str): dataset name.
      encoding (str): feature encoding type.
      training_mode (str): how the model was trained.
      multiclass (bool): multiclass flag.
      seed (int): random seed for reproducibility.
    
    Returns:
      norms (list of float): mean perturbation norms.
      clips (list of float): clipping fractions.
      pcts (list of float): percentage changes.
      accs (list of float): accuracies.
      precisions (list of float): precision scores.
      recalls (list of float): recall scores.
      f1s (list of float): F1 scores.
    """
    
    # Initialize result lists
    results = {
        'accs': [], 'precisions': [], 'recalls': [], 'f1s': [],
        'norms_linf': [], 'norms_l2': [], 'norms_l0': [],
        'pcts_linf': [], 'pcts_l2': [], 'pcts_l0': [],
        'clips': []
    }
    if dataset == 'unsw-nb15':
        attack_params = {
            'fgsm': [{'eps': eps} for eps in [0.0005, 0.001, 0.01, 0.05, 0.1, 0.5, 1]],
            'jsma': [
                {'theta': theta, 'gamma': gamma}
                for theta in [0.0005, 0.001, 0.01, 0.05, 0.1, 0.5, 1]
                for gamma in [0.1]
            ],
            'cw': [
                {'c': c, 'kappa': kappa}
                for c in [0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 100, 1000]
                for kappa in [0]
            ]
        }
    elif dataset == 'mirai':
        attack_params = {
            'fgsm': [{'eps': eps} for eps in [0.0005, 0.001, 0.01, 0.05, 0.1, 0.5, 1]],
            'jsma': [
                {'theta': theta, 'gamma': gamma}
                for theta in [0.0005, 0.001, 0.01, 0.05, 0.1, 0.5, 1]
                for gamma in [0.1]
            ],
            'cw': [
                {'c': c, 'kappa': kappa}
                for c in [0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 100, 1000]
                for kappa in [0]
            ]
        }    

    # Fallback for unknown attack types
    if attack not in attack_params:
        raise ValueError(f"Attack type '{attack}' not supported. Supported types: {list(attack_params.keys())}")
    
    print(f"Running attack: {attack}")

    # Run attacks with appropriate parameters
    for i, params in enumerate(attack_params[attack]):
        start_time = time.time()
        print(f"    - ({i+1}/{len(attack_params[attack])}) Running {attack} with parameters: {params}")
        adv = perform_attack(
            model, all_data, all_labels,
            method=attack,
            params=params
        )
        
        # Evaluate and store results
        metrics = evaluate(model, adv, df_clean, all_labels, encoding)
        acc, prec, rec, f1, mns, cf, pct_changes = metrics

        
        results['accs'].append(acc)
        results['precisions'].append(prec)
        results['recalls'].append(rec)
        results['f1s'].append(f1)
        results['norms_linf'].append(mns['linf'])
        results['norms_l2'].append(mns['l2'])
        results['norms_l0'].append(mns['l0'])
        results['pcts_linf'].append(pct_changes['linf'])
        results['pcts_l2'].append(pct_changes['l2'])
        results['pcts_l0'].append(pct_changes['l0'])
        results['clips'].append(cf)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"    - Time taken: {elapsed_time:.2f} seconds")
    
    return attack_params, results

def sweep_defensive_attack(dataset, attack, encoding, model, all_data, all_labels, df_clean, breakpoints=(3.0, 10.0, 20.0)):
    """
    Run attack with specific parameters against defensively trained models.
    
    Args:
      dataset (str): dataset name ('unsw-nb15' or 'mirai').
      attack (str): attack type ('fgsm', 'cw', 'jsma').
      encoding (str): feature encoding type ('dm', 'stats', 'raw').
      model: the defensively trained model to attack.
      all_data: test data for attacking.
      all_labels: test labels.
      df_clean: clean dataframe for evaluation.
      breakpoints (tuple): perturbation budgets to test (default: 3%, 10%, 20%).
    
    Returns:
      attack_params (dict): parameters used for each attack.
      results (dict): evaluation metrics for each attack parameter.
    """
    
    # Initialize result lists
    results = {
        'accs': [], 'precisions': [], 'recalls': [], 'f1s': [],
        'norms_linf': [], 'norms_l2': [], 'norms_l0': [],
        'pcts_linf': [], 'pcts_l2': [], 'pcts_l0': [],
        'clips': []
    }
    
    # Define parameter lookup dictionary
    attack_param_lookup = {
        'unsw-nb15': {
            'fgsm': {
                'DM': {3.0: 0.0020, 10.0: 0.0102, 20.0: 0.0222},
                'Stats': {3.0: 0.0065, 10.0: 0.0232, 20.0: 0.0477},
                'Raw': {3.0: 0.0076, 10.0: 0.0255, 20.0: 0.0514},
            },
            'cw': {
                'DM': {3.0: 0.2188, 10.0: 1.7038, 20.0: 14.5477},
                'Stats': {3.0: 0.1014, 10.0: 0.3870, 20.0: 13.0739},
                'Raw': {3.0: 0.0984, 10.0: 1.1614, 20.0: 7.2549}
            },
            'jsma': {
                'DM': {3.0: 0.0048, 10.0: 0.0294, 20.0: 0.0722},
                'Stats': {3.0: 0.0178, 10.0: 0.0652, 20.0: 0.1576},
                'Raw': {3.0: 0.0179, 10.0: 0.0599, 20.0: 0.1217}
            }
        },
        'mirai': {
            'fgsm': {
                'DM': {3.0: 0.0024, 10.0: 0.0095, 20.0: 0.0196},
                'Stats': {3.0: 0.0129, 10.0: 0.0438, 20.0: 0.0882},
                'Raw': {3.0: 0.0147, 10.0: 0.0497, 20.0: 0.1015}
            },
            'cw': {
                'DM': {3.0: 2.6329, 10.0: 6.5819, 20.0: 19.1194},
                'Stats': {3.0: 5.7543, 10.0: 29.4873, 20.0: 362.6893},
                'Raw': {3.0: 4.4241, 10.0: 1000, 20.0: 1000}
            },
            'jsma': {
                'DM': {3.0: 0.0054, 10.0: 0.0236, 20.0: 0.0506},
                'Stats': {3.0: 0.0346, 10.0: 0.1189, 20.0: 0.2702},
                'Raw': {3.0: 0.0560, 10.0: 0.2099, 20.0: 0.4501}
            }
        }
    }
    
    # Generate attack parameters based on breakpoints
    attack_params = []
    for bp in breakpoints:
        if attack == 'fgsm':
            eps = attack_param_lookup[dataset][attack][encoding][bp]
            attack_params.append({'eps': eps})
        elif attack == 'jsma':
            theta = attack_param_lookup[dataset][attack][encoding][bp]
            attack_params.append({'theta': theta, 'gamma': 0.1})
        elif attack == 'cw':
            c = attack_param_lookup[dataset][attack][encoding][bp]
            attack_params.append({'c': c, 'kappa': 0})
        else:
            raise ValueError(f"Attack type '{attack}' not supported. Supported types: fgsm, cw, jsma")
    
    print(f"Running attack: {attack} at perturbation budgets {breakpoints}")

    # Run attacks with appropriate parameters
    for i, params in enumerate(attack_params):
        start_time = time.time()
        bp = breakpoints[i]
        print(f"    - ({i+1}/{len(attack_params)}) Running {attack} with parameters: {params} (budget: {bp}%)")
        adv = perform_attack(
            model, all_data, all_labels,
            method=attack,
            params=params
        )
        
        # Evaluate and store results
        metrics = evaluate(model, adv, df_clean, all_labels, encoding)
        acc, prec, rec, f1, mns, cf, pct_changes = metrics
        
        results['accs'].append(acc)
        results['precisions'].append(prec)
        results['recalls'].append(rec)
        results['f1s'].append(f1)
        results['norms_linf'].append(mns['linf'])
        results['norms_l2'].append(mns['l2'])
        results['norms_l0'].append(mns['l0'])
        results['pcts_linf'].append(pct_changes['linf'])
        results['pcts_l2'].append(pct_changes['l2'])
        results['pcts_l0'].append(pct_changes['l0'])
        results['clips'].append(cf)
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"    - Time taken: {elapsed_time:.2f} seconds")
        print(f"    - Results: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}, L2 Norm={mns['l2']:.4f}, Relative L2={pct_changes['l2']:.4f}")
    
    return attack_params, results

def plot_robustness_with_clipping(epsilons, norms, accs, clips, encoding):
    """
    Plot accuracy vs. mean perturbation norm with clipping fraction overlay.

    Args:
      epsilons (list of float): FGSM ε values (for annotation).
      norms (list of float): mean perturbation norms.
      accs (list of float): accuracies.
      clips (list of float): clipping fractions.
      encoding (str): encoding name for title.
    """
    fig, ax1 = plt.subplots(figsize=(6,4))
    ax1.plot(norms, accs, 'b-o', label='Accuracy')
    ax1.set_xlabel('Mean perturbation norm')
    ax1.set_ylabel('Accuracy', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(norms, clips, 'r-s', label='Clipped fraction')
    ax2.set_ylabel('Fraction clipped', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    plt.title(f'FGSM Robustness + Clipping ({encoding})')
    fig.tight_layout()
    plt.show()


def norms_table(norm_objects, epsilons, encodings):
    data = {}
    for encoding, norms in zip(encodings, norm_objects):
        data[encoding] = norms
    df = pd.DataFrame(data, index=epsilons).T
    return df

def pct_change_table(pct_changes, epsilons, encodings):
    data = {}
    for encoding, norms in zip(encodings, pct_changes):
        data[encoding] = norms
    df = pd.DataFrame(data, index=epsilons).T
    return df

def metrics_table(metrics, epsilons, encodings):
    data = {}
    for encoding, metric in zip(encodings, metrics):
        data[encoding] = metric
    df = pd.DataFrame(data, index=epsilons).T
    return df



In [ ]:
import os
import json
import time
import pandas as pd
from datetime import datetime

# Configuration
DATASETS = ['mirai', 'unsw-nb15']
ENCODINGS = ['DM', 'Stats', 'Raw']
TRAINING_MODES = ['baseline', 'pgd', 'mart', 'trades']
ATTACK_TYPES = ['fgsm', 'cw', 'jsma'] 
EPS_VALUES = [0.001, 0.01, 0.1] 
MULTICLASS = [False, True]
SEED = 42

BASE_DIR = 'results/attack_sweep'

In [ ]:
def main(base_dir=BASE_DIR, seed=SEED):

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    os.makedirs(base_dir, exist_ok=True)

    # Total experiments counter
    total_experiments = (
        len(DATASETS) * 
        len(ENCODINGS) * 
        (1 + (len(TRAINING_MODES) - 1) * len(EPS_VALUES)) * 
        len(ATTACK_TYPES)
    )
    completed = 0
    
    # Log file for tracking progress
    log_file = os.path.join(base_dir, f"experiment_log_{timestamp}.txt")
    with open(log_file, 'a') as f:
        f.write(f"Starting experiment run at {datetime.now()}\n")
        f.write(f"Total experiments to run: {total_experiments}\n")

    try:
        for dataset, multiclass in zip(DATASETS, MULTICLASS):
            # Create dataset directory
            dataset_dir = os.path.join(base_dir, dataset)
            os.makedirs(dataset_dir, exist_ok=True)
            
            for encoding in ENCODINGS:
                # First handle baseline (no eps variations)
                baseline_dir = os.path.join(dataset_dir, "baseline")
                os.makedirs(baseline_dir, exist_ok=True)
                
                for attack in ATTACK_TYPES:
                    experiment_id = f"{dataset}_{encoding}_{attack}"

                    print(f"Running experiment {completed+1}/{total_experiments}: {experiment_id}")

                    _training_mode = 'baseline'
                        
                    model, all_data, all_labels, df_clean = load_model_and_data(
                        dataset, encoding, _training_mode, multiclass, seed
                    )
                
                    start_time = time.time()
                    attack_params, results = sweep_attack(
                        dataset=dataset,
                        attack=attack,
                        encoding=encoding,
                        model=model,
                        all_data=all_data,
                        all_labels=all_labels,
                        df_clean=df_clean
                    )
                    
                    print(results)

                    # Create DataFrame and save to CSV
                    df = pd.DataFrame({
                        'parameter_index': range(len(results['accs'])),
                        'attack_params': [str(params) for params in attack_params[attack]],
                        'norm_linf': results['norms_linf'],
                        'norm_l2': results['norms_l2'],
                        'norm_l0': results['norms_l0'],
                        'clip': results['clips'],
                        'pct_change_linf': results['pcts_linf'],
                        'pct_change_l2': results['pcts_l2'],
                        'pct_change_l0': results['pcts_l0'],
                        'accuracy': results['accs'],
                        'precision': results['precisions'],
                        'recall': results['recalls'],
                        'f1': results['f1s']
                    })
                    
                    output_file = os.path.join(baseline_dir, f"{encoding}_{attack}.csv")
                    df.to_csv(output_file, index=False)
                    
                    # Log completion
                    elapsed = time.time() - start_time
                    with open(log_file, 'a') as f:
                        f.write(f"Completed {experiment_id} in {elapsed:.2f} seconds\n")
                        f.write(f"Progress: {completed}/{total_experiments}\n")
                    
                # Handle other training modes with eps variations
                for training_mode in TRAINING_MODES:

                    if training_mode == 'baseline':
                        continue  # Already handled
                    
                    # Create training mode directory
                    training_dir = os.path.join(dataset_dir, training_mode)
                    os.makedirs(training_dir, exist_ok=True)
                    
                    for eps in EPS_VALUES:
                        model_name = f"{training_mode}_eps{eps}"
                        
                        # Create eps-specific directory
                        eps_dir = os.path.join(training_dir, f"eps{eps}")
                        os.makedirs(eps_dir, exist_ok=True)
                        
                        for attack in ATTACK_TYPES:

                            experiment_id = f"{dataset}_{encoding}_{attack}"
                            
                            print(f"Running experiment {completed+1}/{total_experiments}: {experiment_id}")
                            with open(log_file, 'a') as f:
                                f.write(f"Starting {experiment_id} at {datetime.now()}\n")
                            

                            model, all_data, all_labels, df_clean = load_model_and_data(
                                dataset, encoding, model_name, multiclass, SEED
                            )
                        
                            start_time = time.time()
    
                            attack_params, results = sweep_defensive_attack(
                                dataset=dataset,
                                attack=attack,
                                encoding=encoding,
                                model=model,
                                all_data=all_data,
                                all_labels=all_labels,
                                df_clean=df_clean,
                                breakpoints=(3.0, 10.0, 20.0)  # Default, can be omitted
                            )
                            print(results)
                            print(attack_params)
                            
                            # Create DataFrame and save to CSV
                            df = pd.DataFrame({
                                'parameter_index': range(len(results['accs'])),
                                'attack_params': [str(params) for params in attack_params], # [str(params) for params in attack_params[attack]],
                                'norm_linf': results['norms_linf'],
                                'norm_l2': results['norms_l2'],
                                'norm_l0': results['norms_l0'],
                                'clip': results['clips'],
                                'pct_change_linf': results['pcts_linf'],
                                'pct_change_l2': results['pcts_l2'],
                                'pct_change_l0': results['pcts_l0'],
                                'accuracy': results['accs'],
                                'precision': results['precisions'],
                                'recall': results['recalls'],
                                'f1': results['f1s']
                            })
                    
                            output_file = os.path.join(eps_dir, f"{encoding}_{attack}.csv")
                            df.to_csv(output_file, index=False)
                            
                            # Log completion time
                            elapsed = time.time() - start_time
                            with open(log_file, 'a') as f:
                                f.write(f"Completed {experiment_id} in {elapsed:.2f} seconds\n")
                                f.write(f"Progress: {completed}/{total_experiments}\n")
                            
        print(f"All experiments completed! Data saved to {base_dir}")
        with open(log_file, 'a') as f:
            f.write(f"All experiments completed at {datetime.now()}\n")
    
    except Exception as e:
        # Log the error
        with open(log_file, 'a') as f:
            f.write(f"ERROR at {datetime.now()}: {str(e)}\n")
        print(f"Error occurred: {str(e)}")
        print(f"Experiment progress saved: {experiment_id}.")
        raise

if __name__ == "__main__":
    main(base_dir=BASE_DIR, seed=SEED)